In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('IZG20-21data.txt')
winter2 = pd.read_csv ('IZG21-22data.txt')
winter3 = pd.read_csv ('IZG22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [7]:
# eliminate special weather
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 54]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 54]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 54]

In [8]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [9]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [10]:
izg = LocationInfo(
    'Wiscasset', 'Maine', 
    latitude= 44.0214,
    longitude= -70.9749,
    timezone='America/Connecticut'
)

izg

LocationInfo(name='Wiscasset', region='Maine', timezone='America/Connecticut', latitude=44.0214, longitude=-70.9749)

In [11]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(izg.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [12]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(izg.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [13]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(izg.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [14]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:59:29  2020-12-01 16:06:35  2020-12-01
1   2020-12-02 07:00:35  2020-12-02 16:06:17  2020-12-02
2   2020-12-03 07:01:39  2020-12-03 16:06:02  2020-12-03
3   2020-12-04 07:02:42  2020-12-04 16:05:48  2020-12-04
4   2020-12-05 07:03:43  2020-12-05 16:05:38  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:28:55  2021-02-24 17:25:48  2021-02-24
86  2021-02-25 06:27:16  2021-02-25 17:27:08  2021-02-25
87  2021-02-26 06:25:37  2021-02-26 17:28:27  2021-02-26
88  2021-02-27 06:23:57  2021-02-27 17:29:46  2021-02-27
89  2021-02-28 06:22:16  2021-02-28 17:31:04  2021-02-28

[90 rows x 3 columns]


In [15]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:59:13  2021-12-01 16:06:40  2021-12-01
1   2021-12-02 07:00:19  2021-12-02 16:06:21  2021-12-02
2   2021-12-03 07:01:24  2021-12-03 16:06:05  2021-12-03
3   2021-12-04 07:02:27  2021-12-04 16:05:51  2021-12-04
4   2021-12-05 07:03:29  2021-12-05 16:05:40  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:29:18  2022-02-24 17:25:29  2022-02-24
86  2022-02-25 06:27:40  2022-02-25 17:26:48  2022-02-25
87  2022-02-26 06:26:01  2022-02-26 17:28:07  2022-02-26
88  2022-02-27 06:24:21  2022-02-27 17:29:26  2022-02-27
89  2022-02-28 06:22:41  2022-02-28 17:30:45  2022-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:58:57  2022-12-01 16:06:44  2022-12-01
1   2022-12-02 07:00:03  2022-12-02 16:06:25  2022-12-02
2   2022-12-03 07:01:08  2022-12-03 16:06:08  2022-12-03
3   2022-12-04 07:02:12  2022-12-04 16:05:54  2022-12-04
4   2022-12-05 07:03:14  2022-12-05 16:05:42  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:29:42  2023-02-24 17:25:09  2023-02-24
86  2023-02-25 06:28:04  2023-02-25 17:26:29  2023-02-25
87  2023-02-26 06:26:25  2023-02-26 17:27:48  2023-02-26
88  2023-02-27 06:24:46  2023-02-27 17:29:07  2023-02-27
89  2023-02-28 06:23:05  2023-02-28 17:30:26  2023-02-28

[90 rows x 3 columns]


In [17]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [18]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IZG,2020-12-02 21:54:00,0.0001,-SN,2020-12-02,2020-12-02 07:00:35,2020-12-02 16:06:17
1,IZG,2020-12-05 17:54:00,2.5400,-SN BR,2020-12-05,2020-12-05 07:03:43,2020-12-05 16:05:38
2,IZG,2020-12-05 18:54:00,3.3000,SN FG,2020-12-05,2020-12-05 07:03:43,2020-12-05 16:05:38
3,IZG,2020-12-05 19:54:00,3.0500,SN FG,2020-12-05,2020-12-05 07:03:43,2020-12-05 16:05:38
4,IZG,2020-12-05 20:54:00,2.7900,-SN BR,2020-12-05,2020-12-05 07:03:43,2020-12-05 16:05:38
...,...,...,...,...,...,...,...
154,IZG,2021-02-27 14:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46
155,IZG,2021-02-27 15:54:00,1.5200,SN FZFG,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46
156,IZG,2021-02-27 16:54:00,1.7800,SN FZFG,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46
157,IZG,2021-02-27 17:54:00,2.0300,-SN BR,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46


In [19]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IZG,2021-12-02 09:54:00,0.0001,-SN,2021-12-02,2021-12-02 07:00:19,2021-12-02 16:06:21
1,IZG,2021-12-02 10:54:00,1.0200,-SN,2021-12-02,2021-12-02 07:00:19,2021-12-02 16:06:21
2,IZG,2021-12-02 11:54:00,1.2700,-SN,2021-12-02,2021-12-02 07:00:19,2021-12-02 16:06:21
3,IZG,2021-12-02 12:54:00,1.2700,-SN,2021-12-02,2021-12-02 07:00:19,2021-12-02 16:06:21
4,IZG,2021-12-02 13:54:00,1.5200,-SN,2021-12-02,2021-12-02 07:00:19,2021-12-02 16:06:21
...,...,...,...,...,...,...,...
184,IZG,2022-02-25 21:54:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:27:40,2022-02-25 17:26:48
185,IZG,2022-02-25 22:54:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:27:40,2022-02-25 17:26:48
186,IZG,2022-02-25 23:54:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:27:40,2022-02-25 17:26:48
187,IZG,2022-02-26 00:54:00,0.0001,-SN,2022-02-26,2022-02-26 06:26:01,2022-02-26 17:28:07


In [20]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IZG,2022-12-01 18:54:00,0.0001,-SN,2022-12-01,2022-12-01 06:58:57,2022-12-01 16:06:44
1,IZG,2022-12-01 19:54:00,0.0001,-SN,2022-12-01,2022-12-01 06:58:57,2022-12-01 16:06:44
2,IZG,2022-12-12 01:54:00,0.0001,-SN,2022-12-12,2022-12-12 07:09:44,2022-12-12 16:05:31
3,IZG,2022-12-12 02:54:00,0.0001,-SN,2022-12-12,2022-12-12 07:09:44,2022-12-12 16:05:31
4,IZG,2022-12-12 03:54:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:09:44,2022-12-12 16:05:31
...,...,...,...,...,...,...,...
169,IZG,2023-02-26 17:54:00,1.0200,-SN,2023-02-26,2023-02-26 06:26:25,2023-02-26 17:27:48
170,IZG,2023-02-26 18:54:00,0.2500,-SN,2023-02-26,2023-02-26 06:26:25,2023-02-26 17:27:48
171,IZG,2023-02-26 19:54:00,0.0001,-SN,2023-02-26,2023-02-26 06:26:25,2023-02-26 17:27:48
172,IZG,2023-02-26 20:54:00,0.0001,-SN,2023-02-26,2023-02-26 06:26:25,2023-02-26 17:27:48


In [21]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
8,IZG,2020-12-17 09:54:00,0.0001,-SN,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50
9,IZG,2020-12-17 10:54:00,0.5100,SN FZFG,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50
10,IZG,2020-12-17 11:54:00,1.5200,SN FZFG,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50
11,IZG,2020-12-17 12:54:00,1.5200,SN FZFG,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50
12,IZG,2020-12-17 13:54:00,0.7600,-SN BR,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50
...,...,...,...,...,...,...,...
152,IZG,2021-02-27 12:54:00,0.0001,-SN,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46
153,IZG,2021-02-27 13:54:00,0.0001,SN FZFG,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46
154,IZG,2021-02-27 14:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46
155,IZG,2021-02-27 15:54:00,1.5200,SN FZFG,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46


In [22]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [23]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
8,IZG,2020-12-17 09:54:00,0.0001,-SN,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50,2020-12-17 08:13:46,2020-12-17 15:06:50
9,IZG,2020-12-17 10:54:00,0.5100,SN FZFG,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50,2020-12-17 08:13:46,2020-12-17 15:06:50
10,IZG,2020-12-17 11:54:00,1.5200,SN FZFG,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50,2020-12-17 08:13:46,2020-12-17 15:06:50
11,IZG,2020-12-17 12:54:00,1.5200,SN FZFG,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50,2020-12-17 08:13:46,2020-12-17 15:06:50
12,IZG,2020-12-17 13:54:00,0.7600,-SN BR,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50,2020-12-17 08:13:46,2020-12-17 15:06:50
13,IZG,2020-12-17 14:54:00,1.0200,SN FZFG,2020-12-17,2020-12-17 07:13:46,2020-12-17 16:06:50,2020-12-17 08:13:46,2020-12-17 15:06:50
28,IZG,2020-12-29 08:54:00,0.2500,-SN BR,2020-12-29,2020-12-29 07:18:43,2020-12-29 16:13:53,2020-12-29 08:18:43,2020-12-29 15:13:53
33,IZG,2021-01-02 08:54:00,1.2700,-SN BR,2021-01-02,2021-01-02 07:19:03,2021-01-02 16:17:22,2021-01-02 08:19:03,2021-01-02 15:17:22
34,IZG,2021-01-02 09:54:00,0.5100,-SN BR,2021-01-02,2021-01-02 07:19:03,2021-01-02 16:17:22,2021-01-02 08:19:03,2021-01-02 15:17:22
35,IZG,2021-01-02 10:54:00,0.7600,-SN BR,2021-01-02,2021-01-02 07:19:03,2021-01-02 16:17:22,2021-01-02 08:19:03,2021-01-02 15:17:22


In [24]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [25]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [26]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.71064
2021-2022: 0.6694722222222222
2022-2023: 0.47749069767441854


In [27]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IZG,2020-12-02 21:54:00,0.0001,-SN,2020-12-02,2020-12-02 07:00:35,2020-12-02 16:06:17
1,IZG,2020-12-05 17:54:00,2.5400,-SN BR,2020-12-05,2020-12-05 07:03:43,2020-12-05 16:05:38
2,IZG,2020-12-05 18:54:00,3.3000,SN FG,2020-12-05,2020-12-05 07:03:43,2020-12-05 16:05:38
3,IZG,2020-12-05 19:54:00,3.0500,SN FG,2020-12-05,2020-12-05 07:03:43,2020-12-05 16:05:38
4,IZG,2020-12-05 20:54:00,2.7900,-SN BR,2020-12-05,2020-12-05 07:03:43,2020-12-05 16:05:38
...,...,...,...,...,...,...,...
149,IZG,2021-02-22 22:54:00,1.5200,-SN BR,2021-02-22,2021-02-22 06:32:09,2021-02-22 17:23:08
150,IZG,2021-02-22 23:54:00,1.7800,SN FZFG,2021-02-22,2021-02-22 06:32:09,2021-02-22 17:23:08
151,IZG,2021-02-23 00:54:00,1.2700,-SN BR,2021-02-23,2021-02-23 06:30:32,2021-02-23 17:24:28
157,IZG,2021-02-27 17:54:00,2.0300,-SN BR,2021-02-27,2021-02-27 06:23:57,2021-02-27 17:29:46


In [28]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_7672\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [29]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [30]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [31]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.7032017543859649
2021-2022: 0.28025
2022-2023: 0.431642


In [66]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'ME'

data.to_csv('SO.izg.csv', index=False)